<a href="https://colab.research.google.com/github/Hatim-Bhavnagar/Automation_with_Python/blob/main/Send_mail_assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FIlter submitted data for duplicate and write in the seperate files

In [69]:
from excel_tools import read_sheet
import re
import pandas as pd
import openpyxl
from smtp import send
from creds import email, pasw
from validate_clean import clean_data, validate_domain, check_student, remove_duplicates

#def validate_email(regex, email_address):
#    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
#    if(re.fullmatch(regex, email_address)):
#        return True
#    else:
#        return False

def clean_cohort_data(cohortdata):
    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    valids = []
    invalids = []
    #print("cohortdata :", cohortdata)
    for cohort in cohortdata:
    #cleaning the data
      full_name = clean_data(cohort.full_name)
      Moodle_Email = clean_data(cohort.Moodle_Email)

      #Validating the email
      if(re.fullmatch(regex, Moodle_Email)): # validate email as proper format like 'abc@test.com'
        if(validate_domain(Moodle_Email)): #validate email domain is a valid one
          valids.append(cohort)
        else:
          invalids.append(cohort)
      else :  invalids.append(cohort)

    #print(valids)
    #print("Invalids:", invalids)

    return valids, invalids


fields = ['ID', 'Start_time', 'Completion_time', 'User_Email', 'Name', 'Last_modified_time', 'full_name', 'Need_365', 'Mac_or_PC', 'Moodle_Email', 'Attending', 'Alternative','postcode']

cohort1 = read_sheet('sample_data/submissions.xlsx', fields = fields)

cohort2 = read_sheet('sample_data/submissions 2.xlsx', fields = fields)

#print("cohort1:", cohort1)
#print("\n")
#print("cohort2:", cohort2)

valid_cohort1, invalid_cohort1 = clean_cohort_data(cohort1)
valid_cohort2, invalid_cohort2 = clean_cohort_data(cohort2)

#print("\n")
#print("List of Student wtih valid email  from cohort1:")
#print(valid_cohort1)
#print("\n")
#print("List of Student wtih valid email from cohort2:")
#print(valid_cohort2)

cohort1_wo_dup = remove_duplicates(valid_cohort1)
cohort2_wo_dup = remove_duplicates(valid_cohort2)


print("cohort1 final:", cohort1_wo_dup)
print("cohort2 final:", cohort2_wo_dup)

dupes = []
cohort1_final = []
cohort2_final = []

#print("\nCheck cohort1 : ")
for student in cohort1_wo_dup:
  if check_student(student, cohort2_wo_dup) and not check_student(student,dupes):
      dupes.append(student)
  else:
    if not check_student(student,cohort1_final):
      cohort1_final.append(student)

#print("\ndupes :",dupes)
#print("Check cohort2 : /n")
for student in cohort2_wo_dup:
  if not check_student(student, dupes) and not check_student(student,cohort2_final):
    cohort2_final.append(student)

#print("\ncohort1 final:", cohort1_final)
#print("cohort2 final:", cohort2_final)
#print("dupes:", dupes)

for dupe in dupes:
  if len(cohort1_final) < len(cohort2_final):
    cohort1_final.append(dupe)
  else:
    cohort2_final.append(dupe)

print("\ncohort1 final:", cohort1_final)
print("cohort2 final:", cohort2_final)
print("dupes:", dupes)

for sub in cohort1_final:
  send(
      sender = email,
      app_pass = pasw,
      recip = sub.Moodle_Email.upper(),
      sub = "Test Email from Colab",
      body = f"The list of cohort1 are :\n{str(cohort1_final)} \n The list of cohort2 are :\n {str(cohort2_final)} \nThe list of dupes are : \n {str(dupes)}"
  )




cohort1 final: [Row(ID=1, Start_time=datetime.datetime(2025, 8, 5, 22, 9, 1), Completion_time=datetime.datetime(2025, 8, 5, 22, 12, 26), User_Email='anonymous', Name=None, Last_modified_time=None, full_name='Hatim B', Need_365='No', Mac_or_PC='PC', Moodle_Email='hatimb2023@gmail.com', Attending='Yes', Alternative=None, postcode='DD4 6PW'), Row(ID=2, Start_time=datetime.datetime(2025, 8, 5, 22, 12, 29), Completion_time=datetime.datetime(2025, 8, 5, 22, 14, 2), User_Email='anonymous', Name=None, Last_modified_time=None, full_name='Insiya B', Need_365='Yes', Mac_or_PC='MAC', Moodle_Email='insiya2023@gmail.com', Attending='Yes', Alternative=None, postcode='DD2 1RL'), Row(ID=3, Start_time=datetime.datetime(2025, 8, 7, 18, 55, 24), Completion_time=datetime.datetime(2025, 8, 7, 18, 58, 9), User_Email='anonymous', Name=None, Last_modified_time=None, full_name='Aleef Bhavnagar', Need_365='Yes', Mac_or_PC='PC', Moodle_Email='hatimb2211@gmail.com', Attending='No', Alternative='November', postcode